In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
# Import some modules from other libraries
import numpy as np
import random
import torch
import time
import matplotlib.pyplot as plt
from collections import deque

# Import the environment module
from environment import Environment
from qnetworkviz import QNetworkViz

In [ ]:
# The Agent class allows the agent to interact with the environment.
class Agent:

    # The class initialisation function.
    def __init__(self, environment):
        # Set the agent's environment.
        self.environment = environment
        # Create the agent's current state
        self.state = None
        # Create the agent's total reward for the current episode.
        self.total_reward = None
        # Reset the agent.
        self.reset()

    # Function to reset the environment, and set the agent to its initial state. This should be done at the start of every episode.
    def reset(self):
        # Reset the environment for the start of the new episode, and set the agent's state to the initial state as defined by the environment.
        self.state = self.environment.reset()
        # Set the agent's total reward for this episode to zero.
        self.total_reward = 0.0
        

    # Function to make the agent take one step in the environment.
    def step(self,mode,epsilon=None,action=None):
        # Choose an action.
        if mode == "random":
            discrete_action = np.random.randint(4)
            
        elif mode == "egreedy":
            actions = np.arange(0,4)
            probas = np.ones(4)*(epsilon/4)
            probas[action] = 1-epsilon + (epsilon/4)
            discrete_action = np.random.choice(actions,p=probas)
        
        else:
            raise ValueError("mode must be in [random,egreedy]")
            
        # Convert the discrete action into a continuous action.
        continuous_action = self._discrete_action_to_continuous(discrete_action)
        # Take one step in the environment, using this continuous action, based on the agent's current state. This returns the next state, and the new distance to the goal from this new state. It also draws the environment, if display=True was set when creating the environment object..
        next_state, distance_to_goal = self.environment.step(self.state, continuous_action)
        # Compute the reward for this paction.
        reward = self._compute_reward(distance_to_goal)
        # Create a transition tuple for this step.
        transition = (self.state, discrete_action, reward, next_state)
        # Set the agent's state for the next step, as the next state from this step
        self.state = next_state
        # Update the agent's reward for this episode
        self.total_reward += reward
        # Return the transition
        return transition

    # Function for the agent to compute its reward. In this example, the reward is based on the agent's distance to the goal after the agent takes an action.
    def _compute_reward(self, distance_to_goal):
        reward = 1 - distance_to_goal
        return reward

    # Function to convert discrete action (as used by a DQN) to a continuous action (as used by the environment).
    def _discrete_action_to_continuous(self, discrete_action):
        if discrete_action == 0:  # Move right
            continuous_action = np.array([0.1, 0], dtype=np.float32)
        elif discrete_action == 1:#Move down
            continuous_action = np.array([0, -0.1], dtype=np.float32)
        elif discrete_action == 2:#Move left
            continuous_action = np.array([-0.1, 0], dtype=np.float32)
        else :#Move up
            continuous_action = np.array([0, 0.1], dtype=np.float32)
        return continuous_action


In [ ]:
# The Network class inherits the torch.nn.Module class, which represents a neural network.
class Network(torch.nn.Module):

    # The class initialisation function. This takes as arguments the dimension of the network's input (i.e. the dimension of the state), and the dimension of the network's output (i.e. the dimension of the action).
    def __init__(self, input_dimension, output_dimension):
        # Call the initialisation function of the parent class.
        super(Network, self).__init__()
        # Define the network layers. This example network has two hidden layers, each with 100 units.
        self.layer_1 = torch.nn.Linear(in_features=input_dimension, out_features=100)
        self.layer_2 = torch.nn.Linear(in_features=100, out_features=100)
        self.output_layer = torch.nn.Linear(in_features=100, out_features=output_dimension)

    # Function which sends some input data through the network and returns the network's output. In this example, a ReLU activation function is used for both hidden layers, but the output layer has no activation function (it is just a linear layer).
    def forward(self, input):
        layer_1_output = torch.nn.functional.relu(self.layer_1(input))
        layer_2_output = torch.nn.functional.relu(self.layer_2(layer_1_output))
        output = self.output_layer(layer_2_output)
        return output

class ReplayBuffer(deque):
    def __init__(self,min_size):
        super().__init__([],10**6)
        self.min_size = min_size

    def is_full_enough(self):
        return len(self)>=self.min_size

    def get_minibatch(self):
        idx = np.random.choice(np.arange(0,len(self)),self.min_size,replace=False)
        try:
            minibatch = []
            for i in range(len(idx)):
                minibatch += [self[idx[i]]]
        except:
            raise ValueError("Replay Buffer is not full enough")

        return np.array(minibatch)

In [ ]:
# The DQN class determines how to train the above neural network.
class DQN:

    # The class initialisation function.
    def __init__(self):
        # Create a Q-network, which predicts the q-value for a particular state.
        self.q_network = Network(input_dimension=2, output_dimension=4)
        # Define the optimiser which is used when updating the Q-network. The learning rate determines how big each gradient step is during backpropagation.
        self.optimiser = torch.optim.Adam(self.q_network.parameters(), lr=0.001)
        self.target_network = Network(input_dimension=2, output_dimension=4)
        self.target_network.load_state_dict(self.q_network.state_dict())

    # Function that is called whenever we want to train the Q-network. Each call to this function takes in a transition tuple containing the data we use to update the Q-network.
    def train_q_network(self, transition):
        # Set all the gradients stored in the optimiser to zero.
        self.optimiser.zero_grad()
        # Calculate the loss for this transition.
        loss = self._calculate_loss(transition)
        # Compute the gradients based on this loss, i.e. the gradients of the loss with respect to the Q-network parameters.
        loss.backward()
        # Take one gradient step to update the Q-network.
        self.optimiser.step()
        # Return the loss as a scalar
        loss_value = loss.item()
        return loss_value
    
    
    def update_target_network(self):
        parameters_q = torch.nn.Module.state_dict(self.q_network)
        torch.nn.Module.load_state_dict(self.target_network,parameters_q)

    # Function to calculate the loss for a particular transition.
    def _calculate_loss(self,transition):

        gamma = 0.9
        batch_size = len(transition)
        transition = np.array(transition).reshape(batch_size,4)
        
        states_batch = np.array([state.reshape(1,2) for state in transition[:,0]])
        next_states_batch = np.array([state.reshape(1,2) for state in transition[:,3]])

        ### extracting batches array from transition
        
        input_batch =  states_batch.reshape(batch_size,2).astype(np.float32)
        action_batch = transition[:,1].reshape(batch_size,1).astype(np.long)
        reward_batch = transition[:,2].reshape(batch_size,1).astype(np.float32)
        next_states_batch = next_states_batch.reshape(batch_size,2).astype(np.float32)
                
        ### transforming arrays into tensors
        
        input_tensor = torch.tensor(input_batch)
        action_tensor = torch.tensor(action_batch)
        reward_tensor = torch.tensor(reward_batch)
        next_states_tensor = torch.tensor(next_states_batch)

        ### Computing the predicted Q value for state s
        
        q_values = self.q_network.forward(input_tensor)
        q_values = torch.gather(q_values,1,action_tensor)

        ### Computing the actual Q value for this step
        
        #target_q_values = self.q_network.forward(next_states_tensor)
        target_q_values = self.target_network.forward(next_states_tensor)
        target_q_values = torch.gather(target_q_values,1, target_q_values.argmax(1).view(batch_size,1))
        
        predicted_sum_future_rewards = reward_tensor.add(target_q_values*gamma)
        loss = torch.nn.MSELoss()(q_values, predicted_sum_future_rewards)

        return loss
    
    def get_greedy_action(self,state,mode):
        best_reward = -100
        best_action = -100
        
        if mode==0:
            for i in range(4):
                input_state_action = np.append(state,i).reshape(1,3).astype(np.float32)
                input_tensor = torch.tensor(input_state_action)
                predicted_reward = self.q_network.forward(input_tensor)
            
                if predicted_reward>best_reward:
                    best_reward = predicted_reward
                    best_action = i
                    
                    
        elif mode==1:
            input_tensor = torch.tensor(state.reshape(1,2).astype(np.float32))
            q_values = self.q_network.forward(input_tensor)
            best_action = int(q_values.argmax(1).numpy())
            
        else:
            raise ValueError('mode must be either 0 or 1') 
                
        return best_action
    
    def get_qvalues(self,mode):
        
        if mode == 0:
            qvalues = np.zeros((10,10,4))
            for i in range(10):
                for j in range(10):
                    for action in range(4):
                        input_state_action = np.array([(0.05 + i*0.1),(0.05 + j*0.1),action]).reshape(1,3).astype(np.float32)
                        input_tensor = torch.tensor(input_state_action)
                        predicted_qvalue = self.q_network.forward(input_tensor)
                        qvalues[i,j,action] = predicted_qvalue
            return qvalues
        
        elif mode ==1 :
            qvalues = np.zeros((10,10,4))
            for i in range(10):
                for j in range(10):
                    input_state = np.array([(0.05 + i*0.1),(0.05 + j*0.1)]).reshape(1,2).astype(np.float32)
                    input_tensor = torch.tensor(input_state)
                    predicted_qvalues = self.q_network.forward(input_tensor)
                    qvalues[i,j,:] = predicted_qvalues.detach().numpy().reshape(4)
            return qvalues
            
        else:
            raise ValueError('mode must be either 0 or 1')
                

In [ ]:
CID = 799240
np.random.seed(CID)
torch.manual_seed(CID)

environment = Environment(display=False, magnification=500)
agent = Agent(environment)
dqn = DQN()
replay_buffer = ReplayBuffer(min_size=50)
n_steps = 0
loss_vect = []

while n_steps<500:
    # Reset the environment for the start of the episode.
    agent.reset()
    # Loop over steps within this episode. The episode length here is 20.
    for step_num in range(20):
        n_steps+=1
        # Step the agent once, and get the transition tuple for this step
        if not replay_buffer.is_full_enough():
            transition = agent.step('random')
            replay_buffer.append(transition)
        else:
            #best_action = dqn.get_greedy_action(agent.state,mode=1)
            transition = agent.step('random')
            #transition = agent.step('egreedy',epsilon = 1/(1+(n_steps%20)), action=best_action)
            replay_buffer.append(transition)
            mini_batch = replay_buffer.get_minibatch()
            loss = dqn.train_q_network(mini_batch)
            loss_vect+=[loss]
            
    dqn.update_target_network()
            
plt.plot(np.arange(50,n_steps),loss_vect)
plt.xticks(np.arange(50,501,50))
plt.yscale('log')
plt.xlabel('Number of steps')
plt.ylabel('Loss')
plt.title('Value of the loss function at each step of training with target network')
plt.savefig('figs/fig3b.eps')
plt.show()

In [ ]:
### Test greedy policy

test_env = environment = Environment(display=True, magnification=500)
agent = Agent(test_env)
agent.reset()
greedy_path = np.array([agent.state])
time.sleep(1)
for step_num in range(20):
    best_action = dqn.get_greedy_action(agent.state,1)
    transition = agent.step('egreedy',epsilon = 0, action=best_action)
    greedy_path = np.append(greedy_path,[agent.state],axis=0)
    
print(greedy_path)

In [ ]:
viz = QNetworkViz()

In [ ]:
viz.plot_q_values(dqn.get_qvalues(mode=1))
viz.plot_grid()
viz.plot_path(greedy_path)
viz.show()